In [ ]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.optimize import minimize
stops = set(stopwords.words("english"))
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import multiprocessing
import difflib

import sys  

# reload(sys)  
# sys.setdefaultencoding('utf-8')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 1))
#cvect = CountVectorizer(stop_words='english', ngram_range=(1, 1))

tfidf_txt = pd.Series(train['question1'].tolist() + train['question2'].tolist() + test['question1'].tolist() + test['question2'].tolist()).astype(str)

tfidf.fit_transform(tfidf_txt)
#cvect.fit_transform(tfidf_txt)

def diff_ratios(st1, st2):
    seq = difflib.SequenceMatcher()
    seq.set_seqs(str(st1).lower(), str(st2).lower())
    return seq.ratio()

def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

def get_features(df_features):
    print('nouns...')
    df_features['question1_nouns'] = df_features.question1.map(lambda x: [w for w, t in nltk.pos_tag(nltk.word_tokenize(str(x).lower())) if t[:1] in ['N']])
    df_features['question2_nouns'] = df_features.question2.map(lambda x: [w for w, t in nltk.pos_tag(nltk.word_tokenize(str(x).lower())) if t[:1] in ['N']])
    df_features['z_noun_match'] = df_features.apply(lambda r: sum([1 for w in r.question1_nouns if w in r.question2_nouns]), axis=1)  #takes long
    print('lengths...')
    df_features['z_len1'] = df_features.question1.map(lambda x: len(str(x)))
    df_features['z_len2'] = df_features.question2.map(lambda x: len(str(x)))
    df_features['z_word_len1'] = df_features.question1.map(lambda x: len(str(x).split()))
    df_features['z_word_len2'] = df_features.question2.map(lambda x: len(str(x).split()))
    print('difflib...')
    df_features['z_match_ratio'] = df_features.apply(lambda r: diff_ratios(r.question1, r.question2), axis=1)  #takes long
    print('word match...')
    df_features['z_word_match'] = df_features.apply(word_match_share, axis=1, raw=True)
    print('tfidf...')
    df_features['z_tfidf_sum1'] = df_features.question1.map(lambda x: np.sum(tfidf.transform([str(x)]).data))
    df_features['z_tfidf_sum2'] = df_features.question2.map(lambda x: np.sum(tfidf.transform([str(x)]).data))
    df_features['z_tfidf_mean1'] = df_features.question1.map(lambda x: np.mean(tfidf.transform([str(x)]).data))
    df_features['z_tfidf_mean2'] = df_features.question2.map(lambda x: np.mean(tfidf.transform([str(x)]).data))
    df_features['z_tfidf_len1'] = df_features.question1.map(lambda x: len(tfidf.transform([str(x)]).data))
    df_features['z_tfidf_len2'] = df_features.question2.map(lambda x: len(tfidf.transform([str(x)]).data))
    return df_features.fillna(0.0)

train = get_features(train)
#train.to_csv('train.csv', index=False)

# col = [c for c in train.columns if c[:1]=='z']

# pos_train = train[train['is_duplicate'] == 1]
# neg_train = train[train['is_duplicate'] == 0]
# p = 0.165
# scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
# while scale > 1:
#     neg_train = pd.concat([neg_train, neg_train])
#     scale -=1
# neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
# train = pd.concat([pos_train, neg_train])

# x_train, x_valid, y_train, y_valid = train_test_split(train[col], train['is_duplicate'], test_size=0.2, random_state=0)

# params = {}
# params["objective"] = "binary:logistic"
# params['eval_metric'] = 'logloss'
# params["eta"] = 0.02
# params["subsample"] = 0.7
# params["min_child_weight"] = 1
# params["colsample_bytree"] = 0.7
# params["max_depth"] = 4
# params["silent"] = 1
# params["seed"] = 1632

# d_train = xgb.DMatrix(x_train, label=y_train)
# d_valid = xgb.DMatrix(x_valid, label=y_valid)
# watchlist = [(d_train, 'train'), (d_valid, 'valid')]
# bst = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=50, verbose_eval=100) #change to higher #s
# print(log_loss(train.is_duplicate, bst.predict(xgb.DMatrix(train[col]))))

test = get_features(test)
#test.to_csv('test.csv', index=False)

# sub = pd.DataFrame()
# sub['test_id'] = test['test_id']
# sub['is_duplicate'] = bst.predict(xgb.DMatrix(test[col]))

# sub.to_csv('z08_submission_xgb_01.csv', index=False)


nouns...
lengths...
difflib...
word match...


In [ ]:
train.to_csv('owl_feat.csv')

test.to_csv('owl_feat_test.csv')


In [5]:
test

,test_id,question1,question2,question1_nouns,question2_nouns,z_noun_match,z_len1,z_len2,z_word_len1,z_word_len2,z_match_ratio,z_word_match,z_tfidf_sum1,z_tfidf_sum2,z_tfidf_mean1,z_tfidf_mean2,z_tfidf_len1,z_tfidf_len2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,"[surface, compare]","[core, m3, i3, home, surface]",1,57,68,11,14,0.384000,0.266667,2.048751,2.831757,0.409750,0.314640,5,9
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,"[hair, transplant, age]","[cost, require]",0,66,43,14,7,0.440367,0.500000,2.209699,2.151824,0.441940,0.430365,5,5
2,2,What but is the best way to send money from Ch...,What you send money to China?,"[way, money, china]",[money],1,60,29,14,6,0.584270,0.444444,2.174504,1.715722,0.434901,0.571907,5,3
3,3,Which food not emulsifiers?,What foods fibre?,"[food, emulsifiers]",[fibre],0,27,17,4,3,0.545455,0.000000,1.337203,1.399856,0.668602,0.699928,2,2
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,"[aberystwyth, start, reading]",[reading],1,32,30,4,6,0.677419,0.800000,1.620889,1.399301,0.540296,0.699651,3,2
5,5,How are the two wheeler insurance from Bharti ...,I admire I am considering of buying insurance ...,"[wheeler, insurance, bharti, axa, insurance]","[i, i, insurance]",2,60,55,10,10,0.469565,0.200000,1.972337,1.988617,0.493084,0.497154,4,4
6,6,How can I reduce my belly fat through a diet?,How can I reduce my lower belly fat in one month?,"[belly, diet]",[month],0,45,49,10,11,0.702128,0.600000,1.997432,2.229369,0.499358,0.445874,4,5
7,7,"By scrapping the 500 and 1000 rupee notes, how...",How will the recent move to declare 500 and 10...,"[rupee, notes, issue, money]","[move, denomination, illegal, money]",1,98,98,18,17,0.448980,0.363636,3.276648,3.065858,0.297877,0.306586,11,10
8,8,What are the how best books of all time?,What are some of the military history books of...,"[books, time]","[history, books, time]",2,40,56,9,11,0.729167,0.571429,1.697861,1.973717,0.565954,0.493429,3,4
9,9,After 12th years old boy and I had sex with a ...,Can a 14 old guy date a 12 year old girl?,"[years, boy, i, sex, years, girl, consent, any...","[guy, date, year, girl]",1,107,41,22,11,0.432432,0.235294,2.896102,2.558221,0.321789,0.365460,9,7
